# Implementation of the mean Average Precision (mAP) at different intersection over union (IoU)

As described [here](https://www.kaggle.com/c/sartorius-cell-instance-segmentation/overview/evaluation)

Code adapted from [here](https://www.kaggle.com/wcukierski/example-metric-implementation) (please upvote this kernel as well), as the metric should be the same as the 2018 DSB one.

- FIX v4 : Removed background from the analysis
- EDIT v5 : Metric should be computed at image level, I updated comments regarding this.
- EDIT v6, v7 : Variables `false_negatives` and `false_positives` were inverted (?).

In [ ]:
import os
import skimage
import numpy as np
import pandas as pd
import skimage.segmentation
import matplotlib.pyplot as plt

In [ ]:
while(1):
  pass

KeyboardInterrupt: ignored

# Load Example

In [ ]:
def rles_to_mask(encs, shape):
    """
    Decodes a rle.

    Args:
        encs (list of str): Rles for each class.
        shape (tuple [2]): Mask size.

    Returns:
        np array [shape]: Mask.
    """
    img = np.zeros(shape[0] * shape[1], dtype=np.uint)
    for m, enc in enumerate(encs):
        if isinstance(enc, np.float) and np.isnan(enc):
            continue
        enc_split = enc.split()
        for i in range(len(enc_split) // 2):
            start = int(enc_split[2 * i]) - 1
            length = int(enc_split[2 * i + 1])
            img[start: start + length] = 1+m
    return img.reshape(shape)


def rle_decode(mask_rle, shape=(520, 704)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

## Prediction

In [ ]:
import numpy as np
import re
import pycocotools.mask as mask_util
# Read Prediction CSV
ids = pd.read_csv("/content/drive/MyDrive/Kaggle/Sartorius/ensemble/all_val.csv").id
ids = ids.str.replace("train/","").str.replace(".png","").tolist()
gt = pd.read_csv('/content/drive/MyDrive/Kaggle/Sartorius/input/sartorius-cell-instance-segmentation/train.csv')
gt = gt.groupby('id').agg(list).reset_index()

predictions = pd.read_csv("/content/drive/MyDrive/Kaggle/Sartorius/ensemble/submission.csv")
predictions["predicted"] = predictions["predicted"].astype(str).str.replace("[","").str.replace("]","")
predictions = predictions.groupby('id').agg(list).reset_index()

shape = (520,704)
# for col in gt.columns[2:]:
#     gt[col] = gt[col].apply(
#         lambda x: np.unique(x)[0] if len(np.unique(x)) == 1 else np.unique(x)
#     )

scr = []
class_scr = {"shsy5y":[],"astro":[],"cort":[]}
for pic in ids:
  if pic in predictions.id.tolist():
    pred_rles = predictions['predicted'][predictions.id==pic].iloc[0]
    #pred_masks = rles_to_mask(pred_rles, shape).astype(np.uint16)
    pred_masks = [rle_decode(pred_rle, shape).astype(bool) for pred_rle in pred_rles]

    # plt.figure(figsize=(15, 10))
    # plt.imshow(pred_masks)
    # plt.axis(False)
    # plt.show()
    pic_class = gt["cell_type"][gt.id==pic].iloc[0][0]
    gt_rles = gt['annotation'][gt.id==pic].iloc[0]
    #gt_masks = rles_to_mask(gt_rles, shape).astype(np.uint16)
    gt_masks = [rle_decode(gt_rle, shape).astype(bool) for gt_rle in gt_rles]
    # plt.figure(figsize=(15, 10))
    # plt.imshow(gt_masks)
    # plt.axis(False)
    # plt.show()
    # break
    scrr = iou_map(gt_masks , pred_masks, verbose=0)
    class_scr[pic_class].append(scrr)
    scr.append(scrr)
  else:
    scr.append(0)

print("mAP is ", np.mean(scr))
for key,value in class_scr.items():
  print(key,": ",np.mean(value))

mAP is  0.32655559614749435
shsy5y :  0.24723011996999122
astro :  0.20907858812107832
cort :  0.40595539042458034


In [ ]:
mAP is  0.3264759468733958
shsy5y :  0.24583669333706584
astro :  0.21040948690035197
cort :  0.4060211694876772

In [ ]:
mAP is  0.32388232915854576
shsy5y :  0.24280785521827009
astro :  0.20257918135909475
cort :  0.40546347409000333

In [ ]:
mAP is  0.32490849502685976
shsy5y :  0.24624718901720188
astro :  0.21131497689557044
cort :  0.4026329300166886

In [ ]:
mAP is  0.31736504204511373
shsy5y :  0.23969889278104412
astro :  0.19941081438991934
cort :  0.3961266179923819

In [ ]:
mAP is  0.31665147334890553
shsy5y :  0.226252266100146
astro :  0.19928574276152922
cort :  0.40138157085602616


In [ ]:
mAP is  0.161347631897497
shsy5y :  0.08077777824408826
astro :  0.10081415152501094
cort :  0.222432329827901

mAP is  0.314347599736948
shsy5y :  0.2276634523757718
astro :  0.19059365476696755
cort :  0.3995026520076631

Ensemble:

贝叶斯三模型：

mAP is  0.31915768318382876

shsy5y :  0.23813636235883018

astro :  0.2007896165010928

cort :  0.39969022560963


单独三模型：

mAP is  0.31827925420003744

shsy5y :  0.2370447387043947

astro :  0.1971098794508892

cort :  0.39989137715596135

贝叶斯双模型：

mAP is  0.31863701009774936

shsy5y :  0.23811691783601024

astro :  0.2043133417544853

cort :  0.39751712107154824

单独双模型：

mAP is  0.31793259359178233

shsy5y :  0.23718853992009997

astro :  0.20133154496497724

cort :  0.3977149244389089

# Metric

## IoU

In [ ]:
def compute_iou(labels, y_pred):
    """
    Computes the IoU for instance labels and predictions.

    Args:
        labels (np array): Labels.
        y_pred (np array): predictions

    Returns:
        np array: IoU matrix, of size true_objects x pred_objects.
    """

    true_objects = len(np.unique(labels))
    pred_objects = len(np.unique(y_pred))

    # Compute intersection between all objects
    intersection = np.histogram2d(
        labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects)
    )[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins=true_objects)[0]
    area_pred = np.histogram(y_pred, bins=pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection
    iou = intersection / union
    
    return iou[1:, 1:]  # exclude background

## Precision

In [ ]:
def precision_at(threshold, iou):
  try:
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) >= 1  # Correct objects
    false_positives = np.sum(matches, axis=1) == 0  # Extra objects
    false_negatives = np.sum(matches, axis=0) == 0  # Missed objects
    tp, fp, fn = (
        np.sum(true_positives),
        np.sum(false_positives),
        np.sum(false_negatives),
    )
    return tp, fp, fn
  except:
    return 0,1,1

## Overall Metric

### IoU

In [ ]:
def iou_map(truths, preds, verbose=0):
    """
    Computes the metric for the competition.
    Masks contain the segmented pixels where each object has one value associated,
    and 0 is the background.

    Args:
        truths (list of masks): Ground truths.
        preds (list of masks): Predictions.
        verbose (int, optional): Whether to print infos. Defaults to 0.

    Returns:
        float: mAP.
    """
    #iou = compute_iou(truths, preds)

    enc_targs = [mask_util.encode(np.asarray(p, order='F')) for p in truths]
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in preds]
    iou = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    
    #print(ious[0].shape)

    if verbose:
        print("Thresh\tTP\tFP\tFN\tPrec.")

    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)

        p = tp / (tp + fp + fn)
        prec.append(p)

        if verbose:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tps, fps, fns, p))

    if verbose:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))

    return np.mean(prec)